In [ ]:
# Visualize the results from a pde1bvp_coupled_pinn.py run.

In [ ]:
from importlib import import_module
import math as m
import os
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sys

In [ ]:
# Specify the run ID (aka problem name).
runid = "alfven_wave4"

# Add the subdirectory for the run results to the module search path.
run_path = os.path.join(".", runid)
sys.path.append(run_path)

# Import the problem definition from the run results directory.
p = import_module(runid)

# Read the run hyperparameters from the run results directory.
import hyperparameters as hp

In [ ]:
# Load the training points.
xy_train = np.loadtxt(os.path.join(runid, "xy_train.dat"))

In [ ]:
# Load the loss function histories.
losses_model_all = np.loadtxt(os.path.join(runid, "losses_model_all.dat"))
losses_model_bc = np.loadtxt(os.path.join(runid, "losses_model_bc.dat"))
losses_model = np.loadtxt(os.path.join(runid, "losses_model.dat"))
losses_all = np.loadtxt(os.path.join(runid, "losses_all.dat"))
losses_bc = np.loadtxt(os.path.join(runid, "losses_bc.dat"))
losses = np.loadtxt(os.path.join(runid, "losses.dat"))


In [ ]:
# Load the model values.
Y = []
delY = []
for i in range(len(p.variable_names)):
    var_name = p.variable_names[i]
    Y.append(np.loadtxt(os.path.join(runid, "%s_train.dat" % var_name)))
    delY.append(np.loadtxt(os.path.join(runid, "del_%s_train.dat" % var_name)))


In [ ]:
# Plot the loss function histories for the individual models.
n_models = len(p.variable_names)

# Compute the number of rows for the 2-per-row plot.
n_rows = m.ceil(n_models/2)

# Plot the loss history for each model.
plt.figure(figsize=(8, 14))
for i in range(n_models):
    plt.subplot(n_rows, 2, i + 1)
    variable_name = p.variable_names[i]
    plt.semilogy(losses_model_all[:, i], label="$L_{all,%s}$" % variable_name)
    plt.semilogy(losses_model_bc[:, i], label="$L_{bc,%s}$" % variable_name)
    plt.semilogy(losses_model[:, i], label="$L_{%s}$" % variable_name)
    plt.title(variable_name)
    plt.legend()
plt.suptitle("Loss function histories by model")
plt.show()

In [ ]:
# Plot the total loss function history.
plt.semilogy(losses_all, label="$L_{all}$")
plt.semilogy(losses_bc, label="$L_{bc}$")
plt.semilogy(losses, label="$L$")
plt.xlabel("Epoch")
plt.ylabel("Loss function")
plt.legend()
plt.grid()
plt.title("Loss function evolution for %s" % runid)
plt.show()

In [ ]:
# Load the training points.
xt_train = np.loadtxt(os.path.join(runid, "xy_train.dat"))
x_train = xt_train[:, 0]
t_train = xt_train[:, 1]

# Create plot labels from the training points.
x_labels = ["%.1f" % x for x in x_train[0:-1:hp.ny_train]]
t_labels = ["%.1f" % t for t in t_train[0:hp.nx_train]]
t_labels.reverse()

# Plot the model values.
plt.figure(figsize=(12, 6))
for i in range(n_models):
    plt.subplot(n_rows, 2, i + 1)
    variable_name = p.variable_names[i]
    # For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
    Z = np.flip(Y[i].reshape(hp.nx_train, hp.ny_train).T, axis=0)
    ax = sns.heatmap(Z)
    ax.set_xlabel("x")
    ax.set_ylabel("t")
    plt.title(variable_name)
plt.suptitle("Model values")
plt.show()

In [ ]:
# Compute wavenumber and angular frequency.
kx = 2*np.pi/p.wavelength
w = 2*np.pi*p.f

In [ ]:
# Compute analytical solutions for vy and By.
a = 0.1
v1y_a = a*np.sin(kx*x_train - w*t_train)
B1y_a = a*np.sin(kx*x_train - w*t_train + np.pi)

In [ ]:
# Plot the analytical solutions.
plt.figure()

# v1y_a
plt.subplot(2, 1, 1)
xvariable_name = p.variable_names[0]
# For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
Z = np.flip(v1y_a.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z)
ax.set_xlabel("x")
ax.set_ylabel("t")
plt.title(variable_name)

plt.subplot(2, 1, 2)
# B1y_a
Z = np.flip(B1y_a.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z)
ax.set_xlabel("x")
ax.set_ylabel("t")
variable_name = p.variable_names[1]
plt.title(variable_name)

plt.suptitle("Analytical values")
plt.show()

In [ ]:
# Compute the error in the predicted solutions relative to the analytical solutions.
v1y_err = Y[2] - v1y_a
B1y_err = Y[4] - B1y_a

In [ ]:
# Plot the error in the predicted solutions.
plt.figure(figsize=(8, 8))

# v1y_err
plt.subplot(2, 1, 1)
variable_name = p.variable_names[0]
# For a Seaborn heat map, reshape as (n_x, n_y), then transpose, then flip.
Z = np.flip(v1y_err.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z)
ax.set_xlabel("x")
ax.set_ylabel("t")
plt.title(variable_name)

plt.subplot(2, 1, 2)
# B1y_err
Z = np.flip(B1y_err.reshape(hp.nx_train, hp.ny_train).T, axis=0)
ax = sns.heatmap(Z)
ax.set_xlabel("x")
ax.set_ylabel("t")
variable_name = p.variable_names[1]
plt.title(variable_name)

plt.suptitle("Prediction errors")
plt.show()

In [ ]:
it = 0
x = x_train[it::hp.ny_train]
y = Y[2][it::hp.ny_train]
y_a = v1y_a[it::hp.ny_train]
y_err = v1y_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$v_{1y,p}$")
plt.plot(x, y_a, label="$v_{1y,a}$")
plt.plot(x, y_err, label="$v_{1y,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $v_{1y}$ at t = %f, RMS = %f" % (it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$v_{1y}$")
plt.show()

In [ ]:
it = 100
x = x_train[it::hp.ny_train]
y = Y[4][it::hp.ny_train]
y_a = B1y_a[it::hp.ny_train]
y_err = B1y_err[it::hp.ny_train]
rms_err = np.sqrt(np.sum(y_err**2)/len(y_err))
plt.plot(x, y, label="$B_{1y,p}$")
plt.plot(x, y_a, label="$B_{1y,a}$")
plt.plot(x, y_err, label="$B_{1y,err}$")
plt.grid()
plt.legend()
plt.title("Error in predicted $B_{1y}$ at t = %f, RMS = %f" % (it/(hp.ny_train - 1)*(p.t1 - p.t0), rms_err))
plt.xlabel("x")
plt.ylabel("$B_{1y}$")
plt.show()